# Corona Virus Analysiswith SQL

#### The Corona Virus pandemic has had a significant impact on public health and has created an urgent need for data-driven insights to understand the spread of the virus.

#### Instaling all required libraries 

In [52]:
pip install pandasql

Note: you may need to restart the kernel to use updated packages.


#### Importing all required libraries 

In [53]:
import pandas as pd
import pandasql as psql

#### Importing Dataset 

In [54]:
df = pd.read_csv('Corona Virus Dataset.csv')
df

,Province,Country/Region,Latitude,Longitude,Date,Confirmed,Deaths,Recovered
0,Afghanistan,Afghanistan,33.939110,67.709953,22-01-2020,0,0,0
1,Afghanistan,Afghanistan,33.939110,67.709953,23-01-2020,0,0,0
2,Afghanistan,Afghanistan,33.939110,67.709953,24-01-2020,0,0,0
3,Afghanistan,Afghanistan,33.939110,67.709953,25-01-2020,0,0,0
4,Afghanistan,Afghanistan,33.939110,67.709953,26-01-2020,0,0,0
...,...,...,...,...,...,...,...,...
78381,Zimbabwe,Zimbabwe,-19.015438,29.154857,09-06-2021,111,5,161
78382,Zimbabwe,Zimbabwe,-19.015438,29.154857,10-06-2021,64,4,23
78383,Zimbabwe,Zimbabwe,-19.015438,29.154857,11-06-2021,192,3,30
78384,Zimbabwe,Zimbabwe,-19.015438,29.154857,12-06-2021,164,3,22


#### Changing Date formate

In [55]:
df['Date'] = pd.to_datetime(df['Date'], format='%d-%m-%Y')
print(df['Date'])

0       2020-01-22
1       2020-01-23
2       2020-01-24
3       2020-01-25
4       2020-01-26
           ...    
78381   2021-06-09
78382   2021-06-10
78383   2021-06-11
78384   2021-06-12
78385   2021-06-13
Name: Date, Length: 78386, dtype: datetime64[ns]


#### Starting with SQL Queries 

In [56]:
def run_query(query, df):
    return psql.sqldf(query, locals())

Q1: Check for NULL values

In [57]:
query1 = """
SELECT 
    SUM(CASE WHEN Province IS NULL THEN 1 ELSE 0 END) AS Province_nulls,
    SUM(CASE WHEN `Country/Region` IS NULL THEN 1 ELSE 0 END) AS Country_Region_nulls,
    SUM(CASE WHEN Latitude IS NULL THEN 1 ELSE 0 END) AS Latitude_nulls,
    SUM(CASE WHEN Longitude IS NULL THEN 1 ELSE 0 END) AS Longitude_nulls,
    SUM(CASE WHEN Date IS NULL THEN 1 ELSE 0 END) AS Date_nulls,
    SUM(CASE WHEN Confirmed IS NULL THEN 1 ELSE 0 END) AS Confirmed_nulls,
    SUM(CASE WHEN Deaths IS NULL THEN 1 ELSE 0 END) AS Deaths_nulls,
    SUM(CASE WHEN Recovered IS NULL THEN 1 ELSE 0 END) AS Recovered_nulls
FROM df
"""
result_q1 = run_query(query_q1, df)
result_q1

,Province_nulls,Country_Region_nulls,Latitude_nulls,Longitude_nulls,Date_nulls,Confirmed_nulls,Deaths_nulls,Recovered_nulls
0,0,0,0,0,0,0,0,0


Q2: If NULL values are present, update them with zeros for all columns.

We know that there are no NULL values in any of the columns in the dataset. Therefore, there is no need to execute query number 2, which would replace NULL values with zeroes.

Q3: Check total number of rows

In [58]:
query3 = "SELECT COUNT(*) AS total_rows FROM df"
result_q3 = run_query(query_q3, df)
result_q3

,total_rows
0,78386


Q4: Check start date and end date

In [59]:
query4 = "SELECT MIN(Date) AS start_date, MAX(Date) AS end_date FROM df"
result_q4 = run_query(query_q4, df)
result_q4

,start_date,end_date
0,2020-01-22 00:00:00.000000,2021-06-13 00:00:00.000000


Q5: Number of months present in the dataset

In [60]:
query5 = "SELECT COUNT(DISTINCT strftime('%Y-%m', Date)) AS num_months FROM df"
result_q5 = run_query(query_q5, df)
result_q5

,num_months
0,18


Q6: Monthly average for confirmed, deaths, recovered

In [61]:
query6 = """
SELECT 
    strftime('%Y-%m', Date) AS month,
    AVG(Confirmed) AS avg_confirmed,
    AVG(Deaths) AS avg_deaths,
    AVG(Recovered) AS avg_recovered
FROM df
GROUP BY month
"""
result_q6 = run_query(query_q6, df)
result_q6

,month,avg_confirmed,avg_deaths,avg_recovered
0,2020-01,4.145455,0.123377,0.092857
1,2020-02,15.296014,0.593596,7.032020
2,2020-03,161.130289,8.660662,27.873900
3,2020-04,505.800433,41.522294,171.642208
4,2020-05,574.849811,30.280897,318.296397
5,2020-06,859.228139,29.817532,548.791558
6,2020-07,1432.361123,35.109552,983.058232
7,2020-08,1611.842899,37.536657,1299.294721
8,2020-09,1784.587446,34.777273,1438.906710
9,2020-10,2412.199623,36.758274,1420.643067


Q7: Most frequent value for confirmed, deaths, recovered each month

In [62]:
query7 = """
SELECT 
    month,
    Confirmed,
    Deaths,
    Recovered
FROM (
    SELECT 
        strftime('%Y-%m', Date) AS month,
        Confirmed,
        Deaths,
        Recovered,
        ROW_NUMBER() OVER (PARTITION BY strftime('%Y-%m', Date) ORDER BY COUNT(*) DESC) AS r
    FROM df
    GROUP BY month, Confirmed, Deaths, Recovered
) t
WHERE r = 1
"""
result_q7 = run_query(query_q7, df)
result_q7

,month,Confirmed,Deaths,Recovered
0,2020-01,0,0,0
1,2020-02,0,0,0
2,2020-03,0,0,0
3,2020-04,0,0,0
4,2020-05,0,0,0
5,2020-06,0,0,0
6,2020-07,0,0,0
7,2020-08,0,0,0
8,2020-09,0,0,0
9,2020-10,0,0,0


Q8: Minimum values for confirmed, deaths, recovered per year

In [63]:
query8 = """
SELECT 
    strftime('%Y', Date) AS year,
    MIN(Confirmed) AS min_confirmed,
    MIN(Deaths) AS min_deaths,
    MIN(Recovered) AS min_recovered
FROM df
GROUP BY year
"""
result_q8 = run_query(query_q8, df)
result_q8

,year,min_confirmed,min_deaths,min_recovered
0,2020,0,0,0
1,2021,0,0,0


Q9: Maximum values for confirmed, deaths, recovered per year

In [64]:
query9 = """
SELECT 
    strftime('%Y', Date) AS year,
    MAX(Confirmed) AS max_confirmed,
    MAX(Deaths) AS max_deaths,
    MAX(Recovered) AS max_recovered
FROM df
GROUP BY year
"""
result_q9 = run_query(query_q9, df)
result_q9

,year,max_confirmed,max_deaths,max_recovered
0,2020,823225,3752,1123456
1,2021,414188,7374,422436


Q10: Total number of cases of confirmed, deaths, recovered each month

In [65]:
query10 = """
SELECT 
    strftime('%Y-%m', Date) AS month,
    SUM(Confirmed) AS total_confirmed,
    SUM(Deaths) AS total_deaths,
    SUM(Recovered) AS total_recovered
FROM df
GROUP BY month
"""
result_q10 = run_query(query_q10, df)
result_q10

,month,total_confirmed,total_deaths,total_recovered
0,2020-01,6384,190,143
1,2020-02,68312,2651,31405
2,2020-03,769236,41346,133070
3,2020-04,2336798,191833,792987
4,2020-05,2744333,144561,1519547
5,2020-06,3969634,137757,2535417
6,2020-07,6838092,167613,4693120
7,2020-08,7694938,179200,6202833
8,2020-09,8244794,160671,6647749
9,2020-10,11515841,175484,6782150


Q11: Spread of virus with respect to confirmed cases

In [66]:
query11 = """
SELECT 
    SUM(Confirmed) AS total_confirmed,
    AVG(Confirmed) AS avg_confirmed,
    SUM((Confirmed - (SELECT AVG(Confirmed) FROM df)) * (Confirmed - (SELECT AVG(Confirmed) FROM df))) / COUNT(*) AS variance_confirmed
FROM df
"""

def run_query(query, df):
    return psql.sqldf(query, locals())

result_q11 = run_query(query_q11, df)
result_q11['stddev_confirmed'] = (result_q11['variance_confirmed'] ** 0.5)

result_q11

,total_confirmed,avg_confirmed,variance_confirmed,stddev_confirmed
0,169065144,2156.828311,1.572889e+08,12541.488152


Q12: Spread of virus with respect to death cases per month

In [67]:
query12 = """
SELECT 
    strftime('%Y-%m', Date) AS month,
    SUM(Deaths) AS total_deaths,
    AVG(Deaths) AS avg_deaths,
    AVG(Deaths * Deaths) - AVG(Deaths) * AVG(Deaths) AS variance_deaths
FROM df
GROUP BY strftime('%Y-%m', Date)
"""

def run_query(query, df):
    return psql.sqldf(query, locals())

result_q12 = run_query(query_q12, df)

result_q12['stddev_deaths'] = result_q12['variance_deaths'].apply(lambda x: x ** 0.5)

result_q12

,month,total_deaths,avg_deaths,variance_deaths,stddev_deaths
0,2020-01,190,0.123377,4.245817,2.060538
1,2020-02,2651,0.593596,68.321849,8.265703
2,2020-03,41346,8.660662,3900.792265,62.456323
3,2020-04,191833,41.522294,40504.268118,201.256722
4,2020-05,144561,30.280897,20684.911671,143.822501
5,2020-06,137757,29.817532,16929.445710,130.113203
6,2020-07,167613,35.109552,21140.154944,145.396544
7,2020-08,179200,37.536657,23272.996457,152.554897
8,2020-09,160671,34.777273,20102.769224,141.784235
9,2020-10,175484,36.758274,17580.071020,132.589860


Q13: Spread of virus with respect to recovered cases

In [68]:
query13 = """
SELECT 
    SUM(Recovered) AS total_recovered,
    AVG(Recovered) AS avg_recovered,
    SUM((Recovered - (SELECT AVG(Recovered) FROM df)) * (Recovered - (SELECT AVG(Recovered) FROM df))) / COUNT(*) AS variance_recovered
FROM df
"""

def run_query(query, df):
    return psql.sqldf(query, locals())

result_q13 = run_query(query_q13, df)

result_q13['stddev_recovered'] = result_q13['variance_recovered'].apply(lambda x: x ** 0.5)

result_q13

,total_recovered,avg_recovered,variance_recovered,stddev_recovered
0,113089548,1442.726354,1.070295e+08,10345.507395


Q14: Country with highest number of confirmed cases

In [69]:
query14 = """
SELECT `Country/Region` 
FROM df
GROUP BY `Country/Region`
ORDER BY SUM(Confirmed) DESC
LIMIT 1
"""
result_q14 = run_query(query_q14, df)
result_q14

,Country/Region
0,US


Q15: Country with lowest number of death cases

In [70]:
query15 = """
SELECT `Country/Region`
FROM df
GROUP BY `Country/Region`
ORDER BY SUM(Deaths)
LIMIT 1
"""
result_q15 = run_query(query_q15, df)
result_q15

,Country/Region
0,Dominica


Q16: Top 5 countries with highest recovered cases

In [71]:
query16 = """
SELECT `Country/Region`, SUM(Recovered) AS total_recovered
FROM df
GROUP BY `Country/Region`
ORDER BY total_recovered DESC
LIMIT 5
"""
result_q16 = run_query(query_q16, df)
result_q16

,Country/Region,total_recovered
0,India,28089649
1,Brazil,15400169
2,US,6303715
3,Turkey,5202251
4,Russia,4745756
